In [10]:
# Library to create rnn model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Library to read excel data
import pandas as pd 
# Class to transformate data
import controller.transformation as trans
import controller.normalization as norm
import numpy as np

In [11]:
# Read csv data
dirpath = "data/"
filename = "kddcup99_csv.csv"
kddCup = pd.read_csv(dirpath+filename)

In [12]:
# Suffle data
kddCup = kddCup.sample(frac=1)
# Split data in 60% 20% 20%
trainingData, validateData, testData = np.split(kddCup.sample(frac=1), [int(.6*len(kddCup)), int(.8*len(kddCup))])
# Suffle data
trainingData = trainingData.sample(frac=1)
validateData = validateData.sample(frac=1)
testData = testData.sample(frac=1)
print(len(trainingData))

296412


In [13]:
def transformData(kddCup):
    # Transform Text data to number
    protocolTypeLabel = {'icmp': 0, 'tcp': 1, 'udp': 2}
    kddCup["protocol_type"] = transformColumn(kddCup["protocol_type"],protocolTypeLabel,0)
    # print(kddCup["protocol_type"])
    flagLabel = {'OTH': 0, 'REJ': 1, 'RSTO': 2, 'RSTOS0': 3, 'RSTR': 4, 'S0': 5, 'S1': 6, 'S2': 7, 'S3': 8, 'SF': 9, 'SH': 10}
    kddCup["flag"] = transformColumn(kddCup["flag"],flagLabel,0)
    # print(kddCup["flag"])
    transformLabel = {'normal': 0}
    kddCup["label"] = transformColumn(kddCup["label"],transformLabel,1)
    # print(kddCup["label"])
    return kddCup


def transformColumn(column, array, default):
    rows = list(set(column.tolist()))
    for row in rows:
        try:
            column = column.replace([row],array[row])
        except KeyError:
            column = column.replace([row],default)
    return column

In [14]:
def normalizeColumn(data):
    arrayData = np.array(data)
    normalizedData = preprocessing.normalize(arrayData)
    return normalizedData

In [15]:
# Transform text to number
trainingData = transformData(trainingData)
# Split and remove columns
kddCupY = trainingData["label"]
kddCupX = trainingData
kddCupX.pop("label")
kddCupX.pop("service")
# Normalize data
normalizeTrainingDataX = norm.normalizeColumn(kddCupX)
arrayTrainingDataY = np.array(kddCupY)
# Reshape
normalizeTrainingDataX = normalizeTrainingDataX.reshape((normalizeTrainingDataX.shape[0],normalizeTrainingDataX.shape[1],-1))

296412
296412


In [19]:
# Define new Model for rnn
model = tf.keras.models.Sequential()
# Adding model type
model.add(tf.keras.layers.SimpleRNN(8))
# Adding Hidden Layers
model.add(tf.keras.layers.Dense(units=3,activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=5,activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=8,activation='sigmoid'))
# Adding output layer
model.add(tf.keras.layers.Dense(units=2,activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
                  loss=tf.keras.losses.mean_squared_error,
                  metrics=['accuracy'])
epochs = 3

In [20]:
# Train Model
model.fit(normalizeTrainingDataX,arrayTrainingDataY,epochs=epochs,batch_size=25,shuffle=True)

Epoch 1/3
11857/11857 [==============================] - 73s 6ms/step - loss: 0.0342 - accuracy: 0.9173
Epoch 2/3
11857/11857 [==============================] - 67s 6ms/step - loss: 0.0184 - accuracy: 0.8033
Epoch 3/3
11857/11857 [==============================] - 88s 7ms/step - loss: 0.0211 - accuracy: 0.8042


In [18]:
print("\n\nResultados en entrenamiento:")
result = model.predict(normalizeTrainingDataX)
# printStats(getStats(arrayTrainingDAtaY,result))
print(result)



Resultados en entrenamiento:
[[0.9992075  0.9992574 ]
 [0.9992006  0.999251  ]
 [0.9977019  0.9978671 ]
 ...
 [0.9969489  0.9971672 ]
 [0.9977019  0.9978671 ]
 [0.99920905 0.9992589 ]]


In [21]:
# VALIDATE RESULT

# Transform text to number
validateData = trans.transformData(validateData)
# Split and remove columns
kddCupValidateY = validateData["label"]
kddCupValidateX = validateData
kddCupValidateX.pop("label")
kddCupValidateX.pop("service")
# Normalize data
normalizeValidateDataX = norm.normalizeColumn(kddCupValidateX)
arrayValidateDataY = np.array(kddCupValidateY)
# Reshape
normalizeValidateDataX = normalizeValidateDataX.reshape((normalizeValidateDataX.shape[0],normalizeValidateDataX.shape[1],-1))

print("\n\nResultados en validacion")
result = model.predict(normalizeValidateDataX)
print(result)



Resultados en validacion
[[0.03506964 0.03541157]
 [0.98992264 0.99052525]
 [0.9904465  0.9910281 ]
 ...
 [0.9924753  0.99296975]
 [0.9924752  0.9929697 ]
 [0.9924752  0.9929697 ]]
